In [1]:
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time
import random
import os

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [3]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [32, 16498, 9670, 86, 318, 24, 326, 4916, 678, 1914] ['life', 'rez', 'picnic', 'picture', 'shows', 'may', 'dramatic', 'gutwrenching', 'impact', 'holocaust']


In [4]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [5]:
size_layer = 128
dimension_output = len(trainset.target_names)
maxlen = 50
batch_size = 32

In [6]:
class Model:
    def __init__(
        self,
        dict_size,
        size_layers,
        learning_rate,
        num_classes,
        num_blocks = 3,
        block_size = 128,
    ):
        self.X = tf.placeholder(tf.int32,[None,None])
        self.Y = tf.placeholder(tf.int32,[None])
        embeddings = tf.Variable(tf.random_uniform([dict_size, size_layers], -1, 1))
        embedded = tf.nn.embedding_lookup(embeddings, self.X)

        def residual_block(x, size, rate, block):
            with tf.variable_scope(
                'block_%d_%d' % (block, rate), reuse = False
            ):
                conv_filter = tf.layers.conv1d(
                    x,
                    x.shape[2] // 4,
                    kernel_size = size,
                    strides = 1,
                    padding = 'same',
                    dilation_rate = rate,
                    activation = tf.nn.tanh,
                )
                conv_gate = tf.layers.conv1d(
                    x,
                    x.shape[2] // 4,
                    kernel_size = size,
                    strides = 1,
                    padding = 'same',
                    dilation_rate = rate,
                    activation = tf.nn.sigmoid,
                )
                out = tf.multiply(conv_filter, conv_gate)
                out = tf.layers.conv1d(
                    out,
                    block_size,
                    kernel_size = 1,
                    strides = 1,
                    padding = 'same',
                    activation = tf.nn.tanh,
                )
                return tf.add(x, out), out

        forward = tf.layers.conv1d(
            embedded, block_size, kernel_size = 1, strides = 1, padding = 'SAME'
        )
        zeros = tf.zeros_like(forward)
        for i in range(num_blocks):
            for r in [1, 2, 4, 8, 16]:
                forward, s = residual_block(
                    forward, size = 7, rate = r, block = i
                )
                zeros = tf.add(zeros, s)
        forward = tf.layers.conv1d(
            zeros,
            block_size,
            kernel_size = 1,
            strides = 1,
            padding = 'SAME',
            activation = tf.nn.tanh,
        )
        self.logits = tf.reduce_sum(tf.layers.conv1d(
            forward, num_classes, kernel_size = 1, strides = 1, padding = 'SAME'
        ), 1)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=self.logits,
            labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(len(dictionary), size_layer, 1e-3, dimension_output)
sess.run(tf.global_variables_initializer())

In [8]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [10]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   1%|▏         | 4/267 [00:00<00:08, 30.28it/s, accuracy=0.812, cost=0.552]

epoch: 0, pass acc: 0.000000, current acc: 0.551917
time taken: 9.6545729637146
epoch: 0, training loss: 0.841357, training acc: 0.622927, valid loss: 1.017929, valid acc: 0.551917



train minibatch loop:   1%|▏         | 4/267 [00:00<00:08, 30.28it/s, accuracy=0.875, cost=0.569]

epoch: 1, pass acc: 0.551917, current acc: 0.670127
time taken: 9.557583093643188
epoch: 1, training loss: 0.536826, training acc: 0.784307, valid loss: 0.971841, valid acc: 0.670127



train minibatch loop:   1%|▏         | 4/267 [00:00<00:08, 30.16it/s, accuracy=0.812, cost=0.554]

...

time taken: 9.5653977394104
epoch: 5, training loss: 0.089099, training acc: 0.974323, valid loss: 2.461559, valid acc: 0.705311



train minibatch loop:   1%|▏         | 4/267 [00:00<00:08, 30.22it/s, accuracy=1, cost=0.0278]   

time taken: 9.555796384811401
epoch: 6, training loss: 0.096023, training acc: 0.978778, valid loss: 2.703046, valid acc: 0.716541



test minibatch loop: 100%|██████████| 67/67 [00:00<00:00, 96.54it/s, accuracy=0.81, cost=1.53]  

time taken: 9.552894353866577
epoch: 7, training loss: 0.100606, training acc: 0.975613, valid loss: 2.643105, valid acc: 0.712567

break epoch:8



In [11]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x, model.Y: batch_y}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 67/67 [00:00<00:00, 80.99it/s]


In [12]:
print(metrics.classification_report(real_Y, predict_Y, target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.67      0.81      0.73      1058
   positive       0.77      0.61      0.68      1075

avg / total       0.72      0.71      0.71      2133

